In [1]:
import vlc
import tkinter as tk
from tkinter import filedialog, colorchooser
from tkinter import ttk
import threading
import time
import queue
import numpy as np

class MediaPlayer:
    def __init__(self, root):
        self.root = root
        self.root.title("VLC Media Player")
        self.instance = vlc.Instance('--input-repeat=-1', '--fullscreen', '--avcodec-hw=any', '--vout=gl')
        self.player = self.instance.media_player_new()
        self.media_list = []
        self.history = []
        self.update_queue = queue.Queue()
        self.updating_progress = True  
        self.is_day_theme = True  # 默认日间主题
        self.visualizer_color = "#002fa7"  # 默认为克莱因蓝
        self.is_audio_only = False

        self.create_widgets()
        self.apply_theme()  # 应用默认主题

        # 创建并启动一个线程来更新进度
        self.update_thread = threading.Thread(target=self.update_progress_thread, daemon=True)
        self.update_thread.start()

        # 创建并启动一个线程来处理UI更新
        self.ui_update_thread = threading.Thread(target=self.ui_update_thread, daemon=True)
        self.ui_update_thread.start()

        # 绑定窗口关闭事件
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def create_widgets(self):
        # 创建视频帧和控制面板
        self.video_frame = tk.Frame(self.root)
        self.video_frame.pack(fill=tk.BOTH, expand=1)

        self.controls_frame = ttk.Frame(self.root, padding="10 5")
        self.controls_frame.pack(fill=tk.X)
        
        # 控制按钮
        self.play_button = ttk.Button(self.controls_frame, text="Play/Pause", command=self.play_pause)
        self.play_button.grid(row=0, column=0, padx=5)

        self.stop_button = ttk.Button(self.controls_frame, text="Stop", command=self.stop)
        self.stop_button.grid(row=0, column=1, padx=5)

        self.open_button = ttk.Button(self.controls_frame, text="Open", command=self.open_file)
        self.open_button.grid(row=0, column=2, padx=5)

        self.forward_button = ttk.Button(self.controls_frame, text="Forward", command=self.forward)
        self.forward_button.grid(row=0, column=3, padx=5)

        self.backward_button = ttk.Button(self.controls_frame, text="Backward", command=self.backward)
        self.backward_button.grid(row=0, column=4, padx=5)

        # 进度条和时间显示
        self.progress = ttk.Scale(self.controls_frame, orient='horizontal', command=self.set_position)
        self.progress.grid(row=0, column=5, padx=5, sticky="ew")

        self.current_time_label = ttk.Label(self.controls_frame, text="00:00:00")
        self.current_time_label.grid(row=0, column=6, padx=5)

        self.total_time_label = ttk.Label(self.controls_frame, text="00:00:00")
        self.total_time_label.grid(row=0, column=7, padx=5)

         # 历史记录按钮
        self.history_button = ttk.Button(self.controls_frame, text="History", command=self.show_history)
        self.history_button.grid(row=0, column=8, padx=5)

        # 音量控制部分
        self.volume_label = ttk.Label(self.controls_frame, text="Volume")
        self.volume_label.grid(row=0, column=9, padx=5)

        self.volume_slider = ttk.Scale(self.controls_frame, orient='horizontal', from_=0, to=100, command=self.set_volume)
        self.volume_slider.set(100)  
        self.volume_slider.grid(row=0, column=10, padx=5)

        # 让进度条可以伸展
        self.controls_frame.grid_columnconfigure(5, weight=1) 

        # 进度条事件绑定
        self.progress.bind("<Button-1>", self.on_progress_click)
        self.progress.bind("<ButtonRelease-1>", self.on_progress_release)
        self.progress.bind("<B1-Motion>", self.on_progress_drag)

        # 添加主题切换按钮
        self.theme_button = ttk.Button(self.controls_frame, text="Toggle Theme", command=self.toggle_theme)
        self.theme_button.grid(row=0, column=11, padx=5)

        # 添加颜色选择按钮
        self.color_button = ttk.Button(self.controls_frame, text="Choose Color", command=self.choose_color)
        self.color_button.grid(row=0, column=12, padx=5)

        # 添加可视化画布
        self.canvas = tk.Canvas(self.video_frame, bg="black")
        self.canvas.pack(fill=tk.BOTH, expand=1)

    def toggle_theme(self):
        # 切换日间/夜间主题
        self.is_day_theme = not self.is_day_theme
        self.apply_theme()

    def apply_theme(self):
        # 应用日间/夜间主题
        if self.is_day_theme:
            self.root.configure(bg='white')
            self.controls_frame.configure(style='Day.TFrame')
            self.video_frame.configure(bg='white')
            style = ttk.Style()
            style.configure('Day.TFrame', background='white')
            style.configure('TButton', background='lightgray', foreground='black')
            style.configure('TLabel', background='white', foreground='black')
        else:
            self.root.configure(bg='black')
            self.controls_frame.configure(style='Night.TFrame')
            self.video_frame.configure(bg='black')
            style = ttk.Style()
            style.configure('Night.TFrame', background='black')
            style.configure('TButton', background='darkgray', foreground='white')
            style.configure('TLabel', background='black', foreground='white')

    def choose_color(self):
        # 选择可视化颜色
        color = colorchooser.askcolor(initialcolor=self.visualizer_color)[1]
        if color:
            self.visualizer_color = color

    def play_pause(self):
        # 播放/暂停控制
        if self.player.is_playing():
            self.player.pause()
        else:
            self.player.play()

    def stop(self):
        # 停止播放
        self.player.stop()

    def open_file(self):
        # 打开文件对话框并加载媒体文件
        file_path = filedialog.askopenfilename(filetypes=[("Media files", "*.mp4;*.mp3;*.avi;*.mkv")])
        if file_path:
            self.history.append(file_path)
            if len(self.history) > 10:
                self.history.pop(0)
            self.is_audio_only = file_path.lower().endswith(('.mp3', '.wav', '.flac'))
            threading.Thread(target=self.load_media, args=(file_path,)).start()

    def forward(self):
        # 快进10秒
        self.player.set_time(self.player.get_time() + 10000)

    def backward(self):
        # 后退10秒
        self.player.set_time(self.player.get_time() - 10000)

    def set_position(self, pos):
        # 设置播放进度
        if not self.updating_progress:
            self.player.set_position(float(pos) / 1000.0)

    def set_volume(self, volume):
        # 设置音量
        self.player.audio_set_volume(int(float(volume)))

    def update_progress_thread(self):
        # 后台线程，更新播放进度
        while True:
            if self.player.get_length() > 0:
                position = self.player.get_position() * 1000
                current_time = self.player.get_time() // 1000
                total_time = self.player.get_length() // 1000
                self.update_queue.put((position, current_time, total_time))
            time.sleep(1)

    def ui_update_thread(self):
        # 后台线程，更新UI
        while True:
            try:
                position, current_time, total_time = self.update_queue.get(timeout=1)
                self.root.after(0, self.update_progress_ui, position, current_time, total_time)
                if self.is_audio_only and self.player.is_playing():
                    self.root.after(0, self.update_visualizer)
            except queue.Empty:
                pass

    def update_progress_ui(self, position, current_time, total_time):
        # 更新进度条和时间显示
        self.progress.config(to=1000)
        self.progress.set(position)
        self.current_time_label.config(text=self.format_time(current_time))
        self.total_time_label.config(text=self.format_time(total_time))

    def update_visualizer(self):
        # 更新可视化效果
        self.canvas.delete("all")
        width = self.canvas.winfo_width()
        height = self.canvas.winfo_height()
        bars = 300  
        bar_width = width / bars
        for i in range(bars):
            bar_height = np.random.randint(0, height)  # 随机生成柱状图高度
            x0 = i * bar_width
            y0 = height - bar_height
            x1 = (i + 1) * bar_width
            y1 = height
            self.canvas.create_rectangle(x0, y0, x1, y1, fill=self.visualizer_color)

    def show_history(self):
        # 展示历史播放记录
        history_window = tk.Toplevel(self.root)
        history_window.title("History")
        history_list = tk.Listbox(history_window)
        for item in self.history:
            history_list.insert(tk.END, item)
        
        # 绑定双击事件处理函数
        history_list.bind("<Double-1>", lambda event: self.play_from_history(history_list))

        history_list.pack(fill=tk.BOTH, expand=1)

    def play_from_history(self, history_list):
        # 从历史记录中播放选定的文件
        selected_index = history_list.curselection()
        if selected_index:
            selected_file = history_list.get(selected_index)
            self.is_audio_only = selected_file.lower().endswith(('.mp3', '.wav', '.flac'))
            threading.Thread(target=self.load_media, args=(selected_file,)).start()
            history_list.master.destroy()  # 关闭历史记录窗口

    def get_handle(self):
        # 获取视频帧的窗口句柄
        return self.video_frame.winfo_id()

    def load_media(self, file_path):
        # 加载媒体文件并开始播放
        media = self.instance.media_new(file_path)
        self.player.set_media(media)
        self.player.set_hwnd(self.get_handle())
        media.get_mrl()
        self.player.play()

        while self.player.get_state() != vlc.State.Playing:
            time.sleep(0.1)

        self.root.geometry("1200x600")

    def format_time(self, seconds):
        # 格式化时间，将秒数转换为时:分:秒格式
        h = seconds // 3600
        m = (seconds % 3600) // 60
        s = seconds % 60
        return f"{h:02}:{m:02}:{s:02}"

    def on_progress_click(self, event):
        # 进度条被点击时的处理函数
        self.updating_progress = False

    def on_progress_release(self, event):
        # 释放进度条时的处理函数，设置播放位置
        self.updating_progress = True
        self.set_position(self.progress.get())

    def on_progress_drag(self, event):
        # 拖动进度条时的处理函数，设置播放位置
        self.updating_progress = False
        self.set_position(self.progress.get())

    def on_closing(self):
        # 窗口关闭时的处理函数，停止播放并关闭应用
        self.player.stop()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    player = MediaPlayer(root)
    root.geometry("1200x600")
    root.mainloop()

